In [29]:
# import the necessary packages
import tensorflow as tf
import numpy as np
import random
import os
import loadDataset_contrasstive as ldc
import ContrastiveModel as cm

# Define parametes

In [25]:


# model input image size
IMAGE_SIZE = (100,100)

# batch size and the buffer size

BATCH_SIZE = 256
BUFFER_SIZE = BATCH_SIZE * 2

# define autotune
AUTO = tf.data.AUTOTUNE

# define the training parameters
LEARNING_RATE = 0.0001
STEPS_PER_EPOCH = 20
VALIDATION_STEPS = 10
EPOCHS = 5

# define the path to save the model
OUTPUT_PATH = "output"
MODEL_PATH = os.path.join(OUTPUT_PATH, "siamese_network")
OUTPUT_IMAGE_PATH = os.path.join(OUTPUT_PATH, "output_image.png")

epochs = 10
batch_size = 64
margin = 1 

# PairGenerator

In [74]:
train_path="../Training"

val_path="../Test"

train_dataset=ldc.create_dataset(train_path, IMAGE_SIZE, batch_size)
val_dataset=ldc.create_dataset(val_path, IMAGE_SIZE, batch_size)
                     

# Calculate Distance

In [ ]:

def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.keras.backend.sum(tf.keras.backend.square(x - y), axis=1, keepdims=True)
    return tf.keras.backend.sqrt(tf.keras.backend.maximum(sum_square, tf.keras.backend.epsilon()))

# Creating model


In [17]:
input_shape=(100,100,3)
input_1= tf.keras.layers.Input(input_shape)
input_2= tf.keras.layers.Input(input_shape)

embeding_network=create_embeding(input_shape)

tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = tf.keras.layers.Lambda(euclidean_distance, output_shape=(1,))(
    [tower_1, tower_2]
)
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)

# Define the contrastive loss


In [18]:
def loss(margin=1):
    """Provides 'contrastive_loss' an enclosing scope with variable 'margin'.

    Arguments:
        margin: Integer, defines the baseline for distance for which pairs
                should be classified as dissimilar. - (default is 1).

    Returns:
        'contrastive_loss' function with data ('margin') attached.
    """

    def contrastive_loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)


        square_pred = tf.keras.backend.square(y_pred)
        margin_square = tf.keras.backend.square(tf.keras.backend.maximum(margin - y_pred, 0))
        return tf.keras.backend.mean((1 - y_true) * square_pred + (y_true) * margin_square)


    return contrastive_loss

In [26]:
siamese.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])
siamese.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 input_9 (InputLayer)        [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 model_4 (Functional)        (None, 100)                  807408    ['input_8[0][0]']             
                                                                                                  
 model_5 (Functional)        (None, 100)                  807408    ['input_9[0][0]']             
                                                                                            

In [28]:
history = siamese.fit(
    train_dataset,
    validation_data=val_dataset,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_steps=VALIDATION_STEPS,
    epochs=10
)

Epoch 1/10
20/20 [==============================] - 29s 1s/step - loss: 0.1409 - accuracy: 0.8938 - val_loss: 0.1507 - val_accuracy: 0.8313
Epoch 2/10
20/20 [==============================] - 13s 679ms/step - loss: 0.1388 - accuracy: 0.8813 - val_loss: 0.1469 - val_accuracy: 0.8484
Epoch 3/10
20/20 [==============================] - 14s 719ms/step - loss: 0.1336 - accuracy: 0.8906 - val_loss: 0.1378 - val_accuracy: 0.8703
Epoch 4/10
20/20 [==============================] - 13s 648ms/step - loss: 0.1326 - accuracy: 0.8859 - val_loss: 0.1295 - val_accuracy: 0.8625
Epoch 5/10
20/20 [==============================] - 13s 647ms/step - loss: 0.1295 - accuracy: 0.8820 - val_loss: 0.1402 - val_accuracy: 0.8547
Epoch 6/10
20/20 [==============================] - 12s 622ms/step - loss: 0.1290 - accuracy: 0.8828 - val_loss: 0.1423 - val_accuracy: 0.8375
Epoch 7/10
20/20 [==============================] - 12s 632ms/step - loss: 0.1282 - accuracy: 0.8734 - val_loss: 0.1301 - val_accuracy: 0.8547
Ep